In [1]:
import itertools
import httpx
import pandas as pd
import datetime as dt

def find_season_simple(league_id, leagues):
    league = leagues[leagues['id']==league_id]
    name = league['name'].iloc[0]
    iseason = int(league['season'].iloc[0])
    season = f"{iseason+8}/{iseason+9}"
    return name, season

leagues = pd.read_pickle("leagues.pkl")
players = pd.read_pickle("players.pkl")
matches = pd.read_pickle("matches.pkl")

In [2]:
filtered_leagues = leagues[(leagues['game_operation'] == "SBK Ost") | (leagues['game_operation'] == "Floorball Deutschland")]
# sort
filtered_leagues.sort_values(by="season", ascending=False)

,id,operation_id,game_operation,season,name,order_key,link,link_schedule
767,1565,1,Floorball Deutschland,15,Deutsche Meisterschaft U15 Junioren,87,/leagues/1565.json,/leagues/1565/schedule.json
764,1567,1,Floorball Deutschland,15,Deutsche Meisterschaft U17 Junioren,84,/leagues/1567.json,/leagues/1567/schedule.json
766,1572,1,Floorball Deutschland,15,Deutsche Meisterschaft U17 Juniorinnen Kleinfeld,86,/leagues/1572.json,/leagues/1572/schedule.json
768,1566,1,Floorball Deutschland,15,Deutsche Meisterschaft U15 Junioren Kleinfeld,88,/leagues/1566.json,/leagues/1566/schedule.json
769,1568,1,Floorball Deutschland,15,Deutsche Meisterschaft U15 Juniorinnen Kleinfeld,89,/leagues/1568.json,/leagues/1568/schedule.json
...,...,...,...,...,...,...,...,...
1021,160,6,SBK Ost,6,PO U11 mixed,20,/leagues/160.json,/leagues/160/schedule.json
1020,44,6,SBK Ost,6,PO U13 KF weiblich,19,/leagues/44.json,/leagues/44/schedule.json
1019,350,6,SBK Ost,6,PO U13 KF m,18,/leagues/350.json,/leagues/350/schedule.json
1018,85,6,SBK Ost,6,PO U13 GF m,17,/leagues/85.json,/leagues/85/schedule.json


In [3]:
# USV Jena
usvjena = players[(players['team_name'] == 'USV Jena' ) | (players['team_name'] == 'Floorball Pinguine Jena' )] 
# Players only
players_usvjena = usvjena[['first_name','last_name']].drop_duplicates()
for ir,row in players_usvjena.iterrows():
    print(row['first_name'], row['last_name'])

Marie-Charlott Rümmler
Nathalie Berger
Marcus Richter
Ba Duong Dang 
Andre Walther
Sebastian Rau
Andreas Knauft
Lukas Kaul
Manuel Hallabrin
Joshua Reibert
Maximilian Starke
Malin Warnt
Olivia Puls
David  Sindram 
Ulrike Franke
Jerome Thomä
Robert März
Frédéric Schoenfeldt
Tomáš Rauch
Stephan Fahr
Samuel Kaufmann
Milena Pölöskey
Michael Koch
Eric  Hofmann 
Jakob Erdmann
Jonas Appenroth
Kilian Herzog
Hannah Braun
Phuong Linh Dang
Til Schröder
Tobias  Beck
Lukas Holzner
Len Kircher
Helene Weichold
Jonas Pätz
Clemens Weichold
Elias Gerth
Christopher Budras
Lena Zielke
Florian Geifes
Magnus  Nauth
Julian Hofmann
Marius Holzner
Christian Friebe
Anica Schöbe
Stefanie März
Sonja Porsch
Anne Sophie Hagemann
Anna-Viktoria Zerwell
Frauke Lämmer
Lilly Fritsche
Gustav  Brisgen
Johannes  Kah
Zoey Samantha Pook
Mathilda Herold
Angelina Fischer
Elena Haschke
Paulina Jacob
Kevin Gebhardt
Patrick Zimmer
Martin Richter
Benjamin Knobloch
Henriette Schallock
Ronny Bärthel
Helen Wötzel
Carolin Rothhardt
Van

In [4]:
# Sum all leagues for each player
jenalist = []
for ir, jenarow in players_usvjena.iterrows():
    playerstats = players[(players['first_name'] == jenarow['first_name']) & 
                       (players['last_name'] == jenarow['last_name']) & 
                       ((players['team_name'] == 'USV Jena' ) | 
                        (players['team_name'] == 'Floorball Pinguine Jena' ))]

    games_tot = playerstats['games'].sum()
    goals_tot = playerstats['goals'].sum()
    ass_tot = playerstats['assists'].sum()
    penalty_tot = playerstats['penalty_2'].sum()*2 + playerstats['penalty_2and2'].sum()*4 
    + playerstats['penalty_5'].sum()*5 + playerstats['penalty_10'].sum()*10
    + playerstats['penalty_ms_tech'].sum()*20 + playerstats['penalty_ms_full'].sum()*20
    + playerstats['penalty_ms1'].sum()*20 + playerstats['penalty_ms2'].sum()*20
    + playerstats['penalty_ms3'].sum()*20
    points_tot = goals_tot + ass_tot
    
    jenadict = {'first_name': jenarow['first_name'], 'last_name': jenarow['last_name'], 
                'games': int(games_tot), 'goals': int(goals_tot), 'assists': int(ass_tot), 
                'points': int(goals_tot)+int(ass_tot), 'penalty': int(penalty_tot), 
                'goals_per_match': float(goals_tot)/games_tot, 
                'assists_per_match': float(ass_tot)/games_tot, 
                'points_per_match': float(goals_tot+ass_tot)/games_tot, 
                'penalty_per_match': float(penalty_tot)/games_tot
               }
    jenalist.append(jenadict)

jenadf = pd.DataFrame(jenalist)

In [5]:
# Most games
jenadf.sort_values(by="games", ascending=False).iloc[0:10]

,first_name,last_name,games,goals,assists,points,penalty,goals_per_match,assists_per_match,points_per_match,penalty_per_match
0,Marie-Charlott,Rümmler,118,48,38,86,8,0.406780,0.322034,0.728814,0.067797
3,Ba Duong,Dang,118,42,26,68,12,0.355932,0.220339,0.576271,0.101695
7,Lukas,Kaul,108,50,34,84,60,0.462963,0.314815,0.777778,0.555556
6,Andreas,Knauft,94,27,21,48,10,0.287234,0.223404,0.510638,0.106383
20,Samuel,Kaufmann,84,47,53,100,10,0.559524,0.630952,1.190476,0.119048
21,Milena,Pölöskey,79,12,8,20,2,0.151899,0.101266,0.253165,0.025316
10,Maximilian,Starke,78,23,14,37,14,0.294872,0.179487,0.474359,0.179487
5,Sebastian,Rau,75,83,50,133,34,1.106667,0.666667,1.773333,0.453333
22,Michael,Koch,74,21,25,46,8,0.283784,0.337838,0.621622,0.108108
14,Ulrike,Franke,71,0,4,4,0,0.000000,0.056338,0.056338,0.000000


In [6]:
# Most goals
jenadf.sort_values(by="goals", ascending=False).iloc[0:10]

,first_name,last_name,games,goals,assists,points,penalty,goals_per_match,assists_per_match,points_per_match,penalty_per_match
91,Leopold,Oehrlich,63,118,34,152,2,1.873016,0.539683,2.412698,0.031746
5,Sebastian,Rau,75,83,50,133,34,1.106667,0.666667,1.773333,0.453333
103,Fiona,Hartmann,49,78,19,97,2,1.591837,0.387755,1.979592,0.040816
1,Nathalie,Berger,66,64,23,87,6,0.969697,0.348485,1.318182,0.090909
50,Lilly,Fritsche,66,62,20,82,4,0.939394,0.303030,1.242424,0.060606
12,Olivia,Puls,54,55,20,75,6,1.018519,0.370370,1.388889,0.111111
68,Carl Friedrich,Meyer,48,53,24,77,8,1.104167,0.500000,1.604167,0.166667
93,Jonathan,Linn,35,52,19,71,8,1.485714,0.542857,2.028571,0.228571
7,Lukas,Kaul,108,50,34,84,60,0.462963,0.314815,0.777778,0.555556
0,Marie-Charlott,Rümmler,118,48,38,86,8,0.406780,0.322034,0.728814,0.067797


In [7]:
# Most assists
jenadf.sort_values(by="assists", ascending=False).iloc[0:10]

,first_name,last_name,games,goals,assists,points,penalty,goals_per_match,assists_per_match,points_per_match,penalty_per_match
20,Samuel,Kaufmann,84,47,53,100,10,0.559524,0.630952,1.190476,0.119048
5,Sebastian,Rau,75,83,50,133,34,1.106667,0.666667,1.773333,0.453333
0,Marie-Charlott,Rümmler,118,48,38,86,8,0.406780,0.322034,0.728814,0.067797
91,Leopold,Oehrlich,63,118,34,152,2,1.873016,0.539683,2.412698,0.031746
7,Lukas,Kaul,108,50,34,84,60,0.462963,0.314815,0.777778,0.555556
30,Tobias,Beck,55,38,29,67,6,0.690909,0.527273,1.218182,0.109091
3,Ba Duong,Dang,118,42,26,68,12,0.355932,0.220339,0.576271,0.101695
64,Helen,Wötzel,57,28,25,53,6,0.491228,0.438596,0.929825,0.105263
22,Michael,Koch,74,21,25,46,8,0.283784,0.337838,0.621622,0.108108
68,Carl Friedrich,Meyer,48,53,24,77,8,1.104167,0.500000,1.604167,0.166667


In [8]:
# Most points
jenadf.sort_values(by="points", ascending=False).iloc[0:10]

,first_name,last_name,games,goals,assists,points,penalty,goals_per_match,assists_per_match,points_per_match,penalty_per_match
91,Leopold,Oehrlich,63,118,34,152,2,1.873016,0.539683,2.412698,0.031746
5,Sebastian,Rau,75,83,50,133,34,1.106667,0.666667,1.773333,0.453333
20,Samuel,Kaufmann,84,47,53,100,10,0.559524,0.630952,1.190476,0.119048
103,Fiona,Hartmann,49,78,19,97,2,1.591837,0.387755,1.979592,0.040816
1,Nathalie,Berger,66,64,23,87,6,0.969697,0.348485,1.318182,0.090909
0,Marie-Charlott,Rümmler,118,48,38,86,8,0.406780,0.322034,0.728814,0.067797
7,Lukas,Kaul,108,50,34,84,60,0.462963,0.314815,0.777778,0.555556
50,Lilly,Fritsche,66,62,20,82,4,0.939394,0.303030,1.242424,0.060606
68,Carl Friedrich,Meyer,48,53,24,77,8,1.104167,0.500000,1.604167,0.166667
12,Olivia,Puls,54,55,20,75,6,1.018519,0.370370,1.388889,0.111111


In [9]:
# Most penalty minutes
jenadf.sort_values(by="penalty", ascending=False).iloc[0:10]

,first_name,last_name,games,goals,assists,points,penalty,goals_per_match,assists_per_match,points_per_match,penalty_per_match
7,Lukas,Kaul,108,50,34,84,60,0.462963,0.314815,0.777778,0.555556
4,Andre,Walther,36,5,6,11,36,0.138889,0.166667,0.305556,1.000000
5,Sebastian,Rau,75,83,50,133,34,1.106667,0.666667,1.773333,0.453333
18,Tomáš,Rauch,36,42,12,54,28,1.166667,0.333333,1.500000,0.777778
24,Jakob,Erdmann,67,20,18,38,28,0.298507,0.268657,0.567164,0.417910
83,Lukas,Eilenberger,43,39,16,55,22,0.906977,0.372093,1.279070,0.511628
9,Joshua,Reibert,62,44,17,61,20,0.709677,0.274194,0.983871,0.322581
10,Maximilian,Starke,78,23,14,37,14,0.294872,0.179487,0.474359,0.179487
84,Marco,Kraft,43,6,9,15,14,0.139535,0.209302,0.348837,0.325581
130,Jannik,Rex,12,4,1,5,12,0.333333,0.083333,0.416667,1.000000


In [10]:
# Most goals per match
jenadf[jenadf['games']>5].sort_values(by="goals_per_match", ascending=False).iloc[0:10]

,first_name,last_name,games,goals,assists,points,penalty,goals_per_match,assists_per_match,points_per_match,penalty_per_match
91,Leopold,Oehrlich,63,118,34,152,2,1.873016,0.539683,2.412698,0.031746
103,Fiona,Hartmann,49,78,19,97,2,1.591837,0.387755,1.979592,0.040816
93,Jonathan,Linn,35,52,19,71,8,1.485714,0.542857,2.028571,0.228571
32,Len,Kircher,27,40,7,47,0,1.481481,0.259259,1.740741,0.000000
40,Magnus,Nauth,12,15,2,17,0,1.250000,0.166667,1.416667,0.000000
90,Cornelius,Förste,29,34,12,46,0,1.172414,0.413793,1.586207,0.000000
2,Marcus,Richter,12,14,6,20,6,1.166667,0.500000,1.666667,0.500000
18,Tomáš,Rauch,36,42,12,54,28,1.166667,0.333333,1.500000,0.777778
85,Paul,Strickling,16,18,7,25,2,1.125000,0.437500,1.562500,0.125000
5,Sebastian,Rau,75,83,50,133,34,1.106667,0.666667,1.773333,0.453333


In [11]:
# Most assists per match
jenadf[jenadf['games']>5].sort_values(by="assists_per_match", ascending=False).iloc[0:10]

,first_name,last_name,games,goals,assists,points,penalty,goals_per_match,assists_per_match,points_per_match,penalty_per_match
139,Milan,König,20,8,16,24,0,0.400000,0.800000,1.200000,0.000000
117,Jakob Sebastian,Scholz,21,16,15,31,6,0.761905,0.714286,1.476190,0.285714
5,Sebastian,Rau,75,83,50,133,34,1.106667,0.666667,1.773333,0.453333
20,Samuel,Kaufmann,84,47,53,100,10,0.559524,0.630952,1.190476,0.119048
93,Jonathan,Linn,35,52,19,71,8,1.485714,0.542857,2.028571,0.228571
91,Leopold,Oehrlich,63,118,34,152,2,1.873016,0.539683,2.412698,0.031746
23,Eric,Hofmann,28,18,15,33,4,0.642857,0.535714,1.178571,0.142857
30,Tobias,Beck,55,38,29,67,6,0.690909,0.527273,1.218182,0.109091
68,Carl Friedrich,Meyer,48,53,24,77,8,1.104167,0.500000,1.604167,0.166667
51,Gustav,Brisgen,8,2,4,6,0,0.250000,0.500000,0.750000,0.000000


In [12]:
# Most points per match
jenadf[jenadf['games']>5].sort_values(by="points_per_match", ascending=False).iloc[0:10]

,first_name,last_name,games,goals,assists,points,penalty,goals_per_match,assists_per_match,points_per_match,penalty_per_match
91,Leopold,Oehrlich,63,118,34,152,2,1.873016,0.539683,2.412698,0.031746
93,Jonathan,Linn,35,52,19,71,8,1.485714,0.542857,2.028571,0.228571
103,Fiona,Hartmann,49,78,19,97,2,1.591837,0.387755,1.979592,0.040816
5,Sebastian,Rau,75,83,50,133,34,1.106667,0.666667,1.773333,0.453333
32,Len,Kircher,27,40,7,47,0,1.481481,0.259259,1.740741,0.000000
2,Marcus,Richter,12,14,6,20,6,1.166667,0.500000,1.666667,0.500000
68,Carl Friedrich,Meyer,48,53,24,77,8,1.104167,0.500000,1.604167,0.166667
90,Cornelius,Förste,29,34,12,46,0,1.172414,0.413793,1.586207,0.000000
85,Paul,Strickling,16,18,7,25,2,1.125000,0.437500,1.562500,0.125000
18,Tomáš,Rauch,36,42,12,54,28,1.166667,0.333333,1.500000,0.777778


In [13]:
# Most penalty minutes per match
jenadf[jenadf['games']>5].sort_values(by="penalty_per_match", ascending=False).iloc[0:10]

,first_name,last_name,games,goals,assists,points,penalty,goals_per_match,assists_per_match,points_per_match,penalty_per_match
130,Jannik,Rex,12,4,1,5,12,0.333333,0.083333,0.416667,1.000000
4,Andre,Walther,36,5,6,11,36,0.138889,0.166667,0.305556,1.000000
18,Tomáš,Rauch,36,42,12,54,28,1.166667,0.333333,1.500000,0.777778
115,Sonja,Weitzing,7,1,2,3,4,0.142857,0.285714,0.428571,0.571429
7,Lukas,Kaul,108,50,34,84,60,0.462963,0.314815,0.777778,0.555556
83,Lukas,Eilenberger,43,39,16,55,22,0.906977,0.372093,1.279070,0.511628
2,Marcus,Richter,12,14,6,20,6,1.166667,0.500000,1.666667,0.500000
5,Sebastian,Rau,75,83,50,133,34,1.106667,0.666667,1.773333,0.453333
154,Michael,Neumann,9,2,1,3,4,0.222222,0.111111,0.333333,0.444444
24,Jakob,Erdmann,67,20,18,38,28,0.298507,0.268657,0.567164,0.417910


In [15]:
# Overall total number of goals
total_goals = jenadf['goals'].sum()
print(total_goals)

2003


In [17]:
jenamatches = matches[((matches['home_team_name']=="USV Jena") | 
                      (matches['home_team_name']=="Floorball Pinguine Jena") | 
                      (matches['home_team_name']=="SG Saalemädels") | 
                      (matches['guest_team_name']=="USV Jena") | 
                      (matches['guest_team_name']=="Floorball Pinguine Jena") | 
                      (matches['guest_team_name']=="SG Saalemädels")) & (matches['home_goals']>-1)]

jenalist = []
for ir, jenarow in jenamatches.iterrows():
    league, season = find_season_simple(jenarow['league_id'],leagues)
    home = jenarow['home_team_name']
    guest = jenarow['guest_team_name']
    home_goals = jenarow['home_goals']
    guest_goals = jenarow['guest_goals']
    date = jenarow['date']
    
    jenadict = {'league': league, 'season': season, 'home': home, 'guest': guest, 'home_goals': home_goals, 
                'guest_goals': guest_goals, 'date': pd.to_datetime(date)}

    jenalist.append(jenadict)

jenadf = pd.DataFrame(jenalist)
jenadf.sort_values(by="date", ascending=True)

,league,season,home,guest,home_goals,guest_goals,date
388,Stena Line FD-Pokal Herren,15/16,USV Jena,Frankfurt Falcons,3,5,2015-09-20
400,Regionalliga Ost Damen KF,15/16,SSV Heidenau,USV Jena,2,6,2015-10-10
399,Regionalliga Ost Damen KF,15/16,USV Jena,Floor Fighters Chemnitz,3,4,2015-10-10
390,Verbandsliga Ost Herren St. 2,15/16,USV Jena,UHC Sparkasse Weißenfels (D),4,2,2015-10-11
391,Verbandsliga Ost Herren St. 2,15/16,UHC Sparkasse Weißenfels (D),USV Jena,1,4,2015-10-18
...,...,...,...,...,...,...,...
274,Deutsche Meisterschaft U17 Juniorinnen Kleinfeld,23/24,SG Saalemädels,TV Eiche Horn Bremen,12,1,2024-06-01
276,Deutsche Meisterschaft U17 Juniorinnen Kleinfeld,23/24,SG Saalemädels,Floor Fighters Chemnitz,5,4,2024-06-02
275,Deutsche Meisterschaft U17 Juniorinnen Kleinfeld,23/24,Dümptener Füchse,SG Saalemädels,6,4,2024-06-02
298,Damen Kleinfeld Regionalliga Ost,23/24,Floor Fighters Chemnitz,USV Jena,10,5,2024-11-25


In [18]:
jenateams = ["USV Jena","Floorball Pinguine Jena","SG Saalemädels"]
wins = 0
draws = 0
losses = 0
most_goals = 0
opponents = []
gamesplayed = []

for ir, row in jenadf.iterrows():
    if row['home'] in jenateams:
        jenahome = True
    else:
        jenahome = False
        
    # Wins, draws, losses
    if row['home_goals'] == row['guest_goals']:
        draws+=1
    elif ((row['home_goals'] > row['guest_goals']) and jenahome) or ((row['home_goals'] < row['guest_goals']) and not jenahome):
        wins+=1
    else:
        losses+=1        
    
    # Most goals
    if (jenahome and (row['home_goals']>most_goals)):
        most_goals = row['home_goals']
    elif (not jenahome and (row['guest_goals']>most_goals)):
        most_goals = row['guest_goals']
        
    # Most frequent opponent
    if jenahome:
        opp = row['guest']
    else:
        opp = row['home']
    if opp in opponents:
        i = opponents.index(opp)
        gamesplayed[i] += 1
    else:
        opponents.append(opp)
        gamesplayed.append(1)

print(f"Wins: {wins}, Draws: {draws}, Losses: {losses}")
print(f"Percentage Wins: {100*float(wins)/(wins+draws+losses):.0f}%")
print(f"Most goals scored in a match: {most_goals}")

dict = {'opponent': opponents, 'gamesplayed': gamesplayed}
ind_opp = pd.DataFrame(dict)

ind_opp.sort_values(by="opponent", ascending=False)

Wins: 146, Draws: 11, Losses: 318
Percentage Wins: 31%
Most goals scored in a match: 43


,opponent,gamesplayed
1,VfL Red Hocks Kaufering II,1
6,Unihockey Igels Dresden II,7
67,Unihockey Igels Dresden,1
10,UV Zwigge 07,16
72,USV TU Dresden White,2
...,...,...
23,Bautzen Bears,2
52,BSV Markranstädt II,3
53,BSV Markranstädt I,3
26,BSV Markranstädt,10


In [19]:
str_to_find = "Halle"

for ir, row in ind_opp.iterrows():
    if str_to_find in row['opponent']:
        print(row.values)

['SG Landsberg/Halle' 2]
['USV Halle Saalebiber' 22]
['USV Halle Saalebiber II' 3]
